# Story Book Author Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/story_book_author_agents.ipynb)

## Dependencies

In [2]:
# Install dependencies without output
!pip install requests > /dev/null
!pip install praisonai > /dev/null
!pip install openai > /dev/null
!pip install praisonai[crewai] > /dev/null

## Tools

In [3]:
#ToDo: Resolve File Read Issue
import os
import requests
import re
import subprocess
from openai import OpenAI
from crewai_tools import tool
from praisonai_tools import FileReadTool


# Tool to generate an image from a chapter's content and character details
@tool
def GenerateImageTool(chapter_content_and_character_details: str) -> str:
    """
    Generates an image for a given chapter content, including character and location details.
    Uses the OpenAI image generation API, saves it in the current folder, and returns the image path.

    Args:
        chapter_content_and_character_details (str): Description for the image generation.

    Returns:
        str: Path to the saved image.
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    response = client.images.generate(
        model="dall-e-3",
        prompt=f"Image is about: {chapter_content_and_character_details}. Style: Illustration. Create an illustration incorporating a vivid palette with an emphasis on shades of azure and emerald, augmented by splashes of gold for contrast and visual interest. The style should evoke the intricate detail and whimsy of early 20th-century storybook illustrations, blending realism with fantastical elements to create a sense of wonder and enchantment. The composition should be rich in texture, with a soft, luminous lighting that enhances the magical atmosphere. Attention to the interplay of light and shadow will add depth and dimensionality, inviting the viewer to delve into the scene. DON'T include ANY text in this image. DON'T include colour palettes in this image.",
        size="1024x1024",
        quality="standard",
        n=1,
    )

    # Get image URL and save it locally
    image_url = response.data[0].url
    words = chapter_content_and_character_details.split()[:5]
    safe_words = [re.sub(r'[^a-zA-Z0-9_]', '', word) for word in words]
    filename = "_".join(safe_words).lower() + ".png"
    filepath = os.path.join(os.getcwd(), filename)

    # Download the image from the generated URL
    image_response = requests.get(image_url)
    if image_response.status_code == 200:
        with open(filepath, 'wb') as file:
            file.write(image_response.content)
    else:
        print("Failed to download the image.")
        return ""

    return filepath

@tool
def MarkdownToPDFTool(markdownfile_name: str) -> str:
    """
    Converts a Markdown file to a PDF document using the mdpdf command line application.

    Args:
        markdownfile_name (str): Path to the input Markdown file.

    Returns:
        str: Path to the generated PDF file.
    """
    output_file = os.path.splitext(markdownfile_name)[0] + '.pdf'

    # Command to convert markdown to PDF using mdpdf
    cmd = ['mdpdf', '--output', output_file, markdownfile_name]

    # Execute the command and handle errors if any
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error during PDF conversion: {e}")
        return ""

    return output_file

## YAML Prompt

In [ ]:
agent_yaml = """
framework: crewai
topic: "Children's Storybook Creation"
roles:
  story_outliner:
    role: "Story Outliner"
    backstory: "An imaginative creator who lays the foundation of captivating stories for children."
    goal: "Develop an outline for a children's storybook about Animals, including chapter titles and characters for 5 chapters."
    tasks:
      outline_development:
        description: "Create an outline for the children's storybook about Animals, detailing chapter titles and character descriptions for 5 chapters."
        expected_output: "A structured outline document containing 5 chapter titles, with detailed character descriptions and the main plot points for each chapter."
  story_writer:
    role: "Story Writer"
    backstory: "A talented storyteller who brings to life the world and characters outlined, crafting engaging and imaginative tales for children."
    goal: "Write the full content of the story for all 5 chapters, each chapter 100 words, weaving together the narratives and characters outlined."
    tasks:
      story_content_creation:
        description: "Using the outline provided, write the full story content for all chapters, ensuring a cohesive and engaging narrative for children. Each chapter should have 100 words."
        expected_output: "A complete manuscript of the children's storybook about Animals with 5 chapters, each containing approximately 100 words, following the provided outline."
  image_generator:
    role: "Image Generator"
    backstory: "A creative AI specialized in visual storytelling, bringing each chapter to life through imaginative imagery."
    goal: "Generate one image per chapter content provided by the story outliner. Generate a total of 5 images sequentially."
    tasks:
      image_creation:
        description: "Generate 5 images that capture the essence of the children's storybook about Animals, aligning with the themes, characters, and narrative outlined for the chapters."
        expected_output: "A set of 5 digital image files that visually represent the story, incorporating elements from the characters and plot as described in the outline."
    tools:
      - GenerateImageTool
  content_formatter:
    role: "Content Formatter"
    backstory: "A meticulous formatter who enhances the readability and presentation of the storybook."
    goal: "Format the written story content in markdown, including images at the beginning of each chapter."
    tasks:
      content_formatting:
        description: "Format the story content in markdown, including an image at the beginning of each chapter."
        expected_output: "The entire storybook content formatted in markdown, with each chapter title followed by the corresponding image and the chapter content."
    tools:
      - FileReadTool
  markdown_to_pdf_creator:
    role: "PDF Converter"
    backstory: "An efficient converter that transforms Markdown files into professionally formatted PDF documents."
    goal: "Convert the Markdown file to a PDF document. Ensure the file story.md is converted."
    tasks:
      pdf_conversion:
        description: "Convert a Markdown file to a PDF document, ensuring the preservation of formatting, structure, and embedded images using the mdpdf library."
        expected_output: "A PDF file generated from the Markdown input, accurately reflecting the content with proper formatting."
    tools:
      - MarkdownToPDFTool
dependencies: []

"""

## Main

In [ ]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[GenerateImageTool, FileReadTool, MarkdownToPDFTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 2/10


 [2024-11-02 16:26:50][DEBUG]: == Working Agent: Story Outliner
 [2024-11-02 16:26:50][INFO]: == Starting Task: Create an outline for the children's storybook about Animals, detailing chapter titles and character descriptions for 5 chapters.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

**Outline for Children's Storybook: "The Great Animal Adventure"**

**Chapter 1: The Secret of the Enchanted Forest**  
*Character Descriptions:*  
- **Leo the Lion**: A brave and curious lion with a golden mane and a heart of gold. He loves exploring new places and is known for his leadership qualities.  
- **Tilly the Tortoise**: A wise and slow-moving tortoise who knows all the secrets of the forest. She is patient and offers sage advice to her friends.  
- **Benny the Bunny**: A playful and energetic bunny who enjoys hopping around and making friends. He is always up for an adventure and has a great sense of humor.

*Main Plot Points:*  
- Leo, Tilly, an

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

Fail to read the file story_content.txt. Error: [Errno 2] No such file or directory: 'story_content.txt'



APIError: The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.